# Explotary Data Analysis of Hypothesis #1

This notebook analyzes the time spent per move in online chess games and compares patterns across game types (e.g., blitz, bullet, rapid) and outcomes (win/loss). The script processes `.json` files containing game data, extracts move timing info from PGNs, and performs statistical tests.

---


### Imports
These libraries are necessary for data processing, visualization, and statistical analysis.

In [1]:
import json
import re
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

### Flow of the Program
For a more organized fashion, this python script consists of spesific functions to overcome spesific tasks.

### Function: `extract_time_data`
This function is responsible for extract time data. Below is the implementation:

In [2]:
def extract_time_data(pgn_str):
    """Extract move numbers and time spent for each move from PGN string."""
    # Regular expression to match move numbers and clock times
    time_pattern = r'(\d+)\.\s+\{?\[%clk\s+(\d+):(\d+):(\d+\.?\d*)\]\}?'

    # Find all matches in the PGN string
    matches = re.findall(time_pattern, pgn_str)

    if not matches:
        # Try alternative pattern (some PGNs have different format)
        alt_pattern = r'(\d+)\.\s+[a-zA-Z0-9+#=]+\s+\{?\[%clk\s+(\d+):(\d+):(\d+\.?\d*)\]\}?'
        matches = re.findall(alt_pattern, pgn_str)

    move_times = []
    prev_time = None

    for match in matches:
        move_num = int(match[0])
        hours = int(match[1])
        minutes = int(match[2])
        seconds = float(match[3])

        # Convert to total seconds
        total_seconds = hours * 3600 + minutes * 60 + seconds

        if prev_time is not None:
            # Time spent on this move is the difference between previous time and current time
            time_spent = prev_time - total_seconds
            if time_spent > 0:  # Ensure not to get negative times
                move_times.append((move_num, time_spent))

        prev_time = total_seconds

    return move_times

### Function: `parse_json_file`
This function is responsible for parse json file. Below is the implementation:

In [3]:
def parse_json_file(file_path):
    """Parse a JSON file and return game data."""
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)

        games = data.get('games', [])
        print(f"  - Found {len(games)} games in the file")

        # Check the first game structure if available
        if games and len(games) > 0:
            first_game = games[0]
            has_pgn = 'pgn' in first_game
            has_time_control = 'time_control' in first_game
            print(f"  - First game has pgn: {has_pgn}, has time_control: {has_time_control}")

            # Debug: Print more fields from the first game to understand structure
            print(f"  - Available fields in first game: {list(first_game.keys())}")
            if 'white' in first_game:
                print(f"  - White player: {first_game['white']}")
            if 'black' in first_game:
                print(f"  - Black player: {first_game['black']}")
            if 'end_time' in first_game:
                print(f"  - End time: {first_game['end_time']}")

            # Sample the PGN if available
            if has_pgn:
                pgn_sample = first_game['pgn'][:100] + "..." if len(first_game['pgn']) > 100 else first_game['pgn']
                print(f"  - PGN sample: {pgn_sample}")

        return games

    except FileNotFoundError:
        print(f"  - File not found: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"  - Error parsing JSON in file: {file_path}")
        return []
    except Exception as e:
        print(f"  - Unexpected error: {str(e)}")
        return []

### Function: `categorize_game`
This function is responsible for categorize game. Below is the implementation:

In [4]:
def categorize_game(time_control):
    """Categorize a game based on its time control."""
    if not time_control:
        return "unknown"

    # For daily games or games with days (e.g., "1/604800")
    if isinstance(time_control, str) and time_control.startswith("1/") and int(time_control.split('/')[1]) >= 86400:
        return "daily"

    # Extract the base time in seconds
    try:
        base_time = int(time_control.split('+')[0]) if isinstance(time_control, str) and '+' in time_control else int(time_control)
    except (ValueError, TypeError):
        return "unknown"

    if base_time < 180:  # Less than 3 minutes
        return "bullet"
    elif base_time < 600:  # Less than 10 minutes
        return "blitz"
    else:
        return "rapid"

### Function: `determine_game_result`
This function is responsible for determine game result. Below is the implementation:

In [5]:
def determine_game_result(game):
    """Determine if mreko0 won or lost the game."""
    # We are analyzing in the scope of my game results only.
    MY_USERNAME = "MrEKO0"

    # Debug the game structure
    if 'white' in game:
        white_player = game['white']
        print(f"  - White player: {white_player}")
    if 'black' in game:
        black_player = game['black']
        print(f"  - Black player: {black_player}")

    # First check if I am white or black in the spesific game.
    your_color = None
    opponent_color = None

    # Method 1: Check for direct string username
    if 'white' in game and game['white'] == MY_USERNAME:
        your_color = 'white'
        opponent_color = 'black'
    elif 'black' in game and game['black'] == MY_USERNAME:
        your_color = 'black'
        opponent_color = 'white'
    # Method 2: Check for dictionary with username
    elif 'white' in game and isinstance(game['white'], dict) and game['white'].get('username') == MY_USERNAME:
        your_color = 'white'
        opponent_color = 'black'
    elif 'black' in game and isinstance(game['black'], dict) and game['black'].get('username') == MY_USERNAME:
        your_color = 'black'
        opponent_color = 'white'

    if not your_color:
        print(f"  - Could not find {MY_USERNAME} as white or black player")
        return 'other'

    print(f"  - Found {MY_USERNAME} as {your_color} player")

    # Look for game ending condition
    if 'end_time' in game:
        print(f"  - Game ended at: {game['end_time']}")

    # Method 1: Check for an explicit user win field
    if your_color + '_result' in game:
        result = game[your_color + '_result'].lower()
        print(f"  - {your_color}_result field: {result}")
        if result == 'win':
            return 'win'
        else:
            return 'loss'

    # Method 2: Check for explicit end reason
    end_reason_fields = ['state', 'status', 'end_reason', 'termination']
    for field in end_reason_fields:
        if field in game:
            end_reason = str(game[field]).lower()
            print(f"  - {field}: {end_reason}")

            # Win conditions
            if end_reason in ['win', 'mate', 'checkmate'] and 'winner' in game and game['winner'] == your_color:
                return 'win'
            if opponent_color + '_checkmated' in end_reason or opponent_color + '_resigned' in end_reason:
                return 'win'
            if your_color + '_won' in end_reason or your_color + '_wins' in end_reason:
                return 'win'

            # Loss conditions
            if end_reason in ['checkmated', 'resigned', 'timeout', 'abandoned']:
                if 'winner' in game and game['winner'] == opponent_color:
                    return 'loss'
                # If no winner field but these usually mean loss
                return 'loss'
            if your_color + '_checkmated' in end_reason or your_color + '_resigned' in end_reason:
                return 'loss'
            if your_color + '_lost' in end_reason:
                return 'loss'

    # Method 3: Check for direct winner field
    if 'winner' in game:
        winner = str(game['winner']).lower()
        print(f"  - Winner field: {winner}")
        if winner == your_color:
            return 'win'
        elif winner == opponent_color:
            return 'loss'
        elif winner == MY_USERNAME.lower():
            return 'win'

    # Method 4: Check PGN for result
    pgn = game.get('pgn', '')
    if f'[White "{MY_USERNAME}"]' in pgn and '[Result "1-0"]' in pgn:
        return 'win'
    elif f'[Black "{MY_USERNAME}"]' in pgn and '[Result "0-1"]' in pgn:
        return 'win'
    elif f'[White "{MY_USERNAME}"]' in pgn and '[Result "0-1"]' in pgn:
        return 'loss'
    elif f'[Black "{MY_USERNAME}"]' in pgn and '[Result "1-0"]' in pgn:
        return 'loss'

    # Method 5: For Chess.com format, specifically check for a result field
    if 'white' in game and isinstance(game['white'], dict) and 'result' in game['white']:
        white_result = game['white']['result'].lower()
        if your_color == 'white':
            if white_result == 'win':
                return 'win'
            else:
                return 'loss'
        else:  # Black
            if white_result == 'win':
                return 'loss'
            else:
                return 'win'

    if 'black' in game and isinstance(game['black'], dict) and 'result' in game['black']:
        black_result = game['black']['result'].lower()
        if your_color == 'black':
            if black_result == 'win':
                return 'win'
            else:
                return 'loss'
        else:  # White
            if black_result == 'win':
                return 'loss'
            else:
                return 'win'

    # If we still cannot determine, look at game details one last time
    if 'details' in game and isinstance(game['details'], str):
        details = game['details'].lower()
        print(f"  - Game details: {details}")
        if f"{MY_USERNAME.lower()} won" in details:
            return 'win'
        elif "timeout" in details and your_color in details:
            return 'loss'
        elif "resigned" in details and your_color in details:
            return 'loss'

    # If we cannot determine or the game was a draw
    print("  - Could not determine game result, marking as 'other'")
    return 'other'  # 'other' includes draws and indeterminate results

### Function: `main`
This is where we define the main function. It checks for available JSON files, processes each game, and generates visualizations and  if enabled, statistical reports.

In [6]:
def main(json_files):
    # Initialize containers for move times categorized by game type and result
    results = {
        'all': {'win': [], 'loss': []},
        'blitz': {'win': [], 'loss': []},
        'bullet': {'win': [], 'loss': []},
        'rapid': {'win': [], 'loss': []},
        'daily': {'win': [], 'loss': []},
    }

    total_games_with_data = 0
    games_won = 0
    games_lost = 0
    games_other = 0

    # Process each JSON file
    for file_name in json_files:
        if os.path.exists(file_name):
            print(f"Processing file: {file_name}")
            games = parse_json_file(file_name)

            games_with_data_in_file = 0
            file_wins = 0
            file_losses = 0
            file_other = 0

            for i, game in enumerate(games):
                if i < 3:  # Debug first 3 games
                    print(f"\nExamining game {i+1}:")

                pgn = game.get('pgn', '')
                time_control = game.get('time_control', '')

                # Skip games without PGN or time control
                if not pgn or not time_control:
                    if i < 3:
                        print(f"  - Skipping game {i+1}: Missing PGN or time control")
                    continue

                # Extract move times
                move_times = extract_time_data(pgn)

                if move_times:
                    games_with_data_in_file += 1

                    # Determine game result
                    result = determine_game_result(game)

                    if i < 3:  # Debug first 3 games
                        print(f"  - Game {i+1} result: {result}")

                    if result == 'win':
                        games_won += 1
                        file_wins += 1
                    elif result == 'loss':
                        games_lost += 1
                        file_losses += 1
                    else:
                        games_other += 1
                        file_other += 1
                        continue  # Skip draws and indeterminate games

                    # Categorize by game type
                    game_type = categorize_game(time_control)

                    # Add to appropriate lists
                    if result in ['win', 'loss']:
                        results['all'][result].extend(move_times)

                        if game_type in ['blitz', 'bullet', 'rapid', 'daily']:
                            results[game_type][result].extend(move_times)

            print(f"  - Extracted timing data from {games_with_data_in_file} games")
            print(f"  - In this file: Wins: {file_wins}, Losses: {file_losses}, Other: {file_other}")
            total_games_with_data += games_with_data_in_file

        else:
            print(f"File not found: {file_name}")

    # Print statistics
    print("\nSUMMARY:")
    print(f"Total games with timing data: {total_games_with_data}")
    print(f"Games won: {games_won}")
    print(f"Games lost: {games_lost}")
    print(f"Games drawn/indeterminate: {games_other}")

    print("\nMove data points:")
    for category in results:
        for result in ['win', 'loss']:
            move_count = len(results[category][result])
            print(f"  - {category.capitalize()} games - {result}: {move_count} moves")

    # Generate plots for each category and result
    for category in results:
        for result in ['win', 'loss']:
            move_times = results[category][result]
            if move_times:
                title = f"{category.capitalize()} Games - {result.capitalize()}"
                filename = f"{category}_{result}.png"
                plot_move_times(move_times, title, filename)
                print(f"Generated plot for {title} with {len(move_times)} move data points")
            else:
                print(f"No move time data found for {category} games - {result}")

    # Create combined plots for direct comparison
    create_combined_plots(results)

    create_additional_plots(results)

    # perform_hypothesis_tests(results) Uncomment to use the function when defined.

### Function: `create_combined_plots`
This function is responsible for create combined plots. Below is the implementation:

In [7]:
def create_combined_plots(results):
    """Create separate plots for wins and losses for each category."""
    categories = ['all', 'blitz', 'bullet', 'rapid', 'daily']

    for category in categories:
        win_data = results[category]['win']
        loss_data = results[category]['loss']

        # Create separate plot for wins
        if win_data:
            plt.figure(figsize=(14, 10))
            win_moves = [mt[0] for mt in win_data]
            win_times = [mt[1] for mt in win_data]
            plt.scatter(win_moves, win_times, alpha=0.5, color='green', label='Wins', s=30)

            # Add trendline for wins
            z_win = np.polyfit(win_moves, win_times, 1)
            p_win = np.poly1d(z_win)
            plt.plot(sorted(win_moves), p_win(sorted(win_moves)), "g--", alpha=0.7)

            # Calculate and display average time per move for wins
            avg_win_time = sum(win_times) / len(win_times)
            plt.annotate(f'Avg time: {avg_win_time:.2f}s',
                         xy=(0.05, 0.95), xycoords='axes fraction',
                         fontsize=12, color='green')

            # Add labels and title
            plt.xlabel('Move Number')
            plt.ylabel('Time Spent (seconds)')
            plt.title(f'Time Spent per Move - {category.capitalize()} Games (Wins)')

            plt.legend()
            plt.grid(True, alpha=0.3)

            # Set axis limits for better visualization
            plt.ylim(0, min(max(win_times) * 1.1, 120))  # Cap at 120 seconds for readability

            # Save the plot
            filename = f"{category}_wins.png"
            plt.savefig(filename)
            plt.close()
            print(f"Wins plot saved as {filename}")

        # Create separate plot for losses
        if loss_data:
            plt.figure(figsize=(14, 10))
            loss_moves = [mt[0] for mt in loss_data]
            loss_times = [mt[1] for mt in loss_data]
            plt.scatter(loss_moves, loss_times, alpha=0.5, color='red', label='Losses', s=30)

            # Add trendline for losses
            z_loss = np.polyfit(loss_moves, loss_times, 1)
            p_loss = np.poly1d(z_loss)
            plt.plot(sorted(loss_moves), p_loss(sorted(loss_moves)), "r--", alpha=0.7)

            # Calculate and display average time per move for losses
            avg_loss_time = sum(loss_times) / len(loss_times)
            plt.annotate(f'Avg time: {avg_loss_time:.2f}s',
                         xy=(0.05, 0.95), xycoords='axes fraction',
                         fontsize=12, color='red')

            # Add labels and title
            plt.xlabel('Move Number')
            plt.ylabel('Time Spent (seconds)')
            plt.title(f'Time Spent per Move - {category.capitalize()} Games (Losses)')

            plt.legend()
            plt.grid(True, alpha=0.3)

            # Set axis limits for better visualization
            plt.ylim(0, min(max(loss_times) * 1.1, 120))  # Cap at 120 seconds for readability

            # Save the plot
            filename = f"{category}_losses.png"
            plt.savefig(filename)
            plt.close()
            print(f"Losses plot saved as {filename}")

### Function: `plot_move_times`
This function is responsible for plot move times. Below is the implementation:

In [8]:
def plot_move_times(move_times, title, filename):
    """Create a scatter plot of move times."""
    if not move_times:
        print(f"No data for {title}")
        return

    move_numbers = [mt[0] for mt in move_times]
    times_spent = [mt[1] for mt in move_times]

    plt.figure(figsize=(12, 8))

    # Create scatter plot
    plt.scatter(move_numbers, times_spent, alpha=0.5, s=30)

    # Add trendline
    z = np.polyfit(move_numbers, times_spent, 1)
    p = np.poly1d(z)
    plt.plot(move_numbers, p(move_numbers), "r--", alpha=0.7)

    # Add labels and title
    plt.xlabel('Move Number')
    plt.ylabel('Time Spent (seconds)')
    plt.title(f'Time Spent per Move - {title}')

    # Add average time per move
    avg_time = sum(times_spent) / len(times_spent)
    plt.axhline(avg_time, color='blue', linestyle='--', alpha=0.7)
    plt.annotate(f'Average: {avg_time:.2f}s',
                xy=(0.05, 0.95), xycoords='axes fraction',
                fontsize=12, color='blue')

    # Add grid
    plt.grid(True, alpha=0.3)

    # Set axis limits for better visualization
    plt.ylim(0, min(max(times_spent) * 1.1, 120))  # Cap at 120 seconds for readability

    # Save the plot
    plt.savefig(filename)
    plt.close()
    print(f"Plot saved as {filename}")

### Function: `create_additional_plots`
This function is responsible for create additional plots. Below is the implementation:

In [9]:
def create_additional_plots(results):
    """Create additional plots (heatmap, violin, histogram, box plot) for each category and result."""
    categories = ['all', 'blitz', 'bullet', 'rapid', 'daily']
    results_types = ['win', 'loss']

    for category in categories:
        # Skip if either wins or losses are empty
        if not results[category]['win'] or not results[category]['loss']:
            print(f"Skipping additional plots for {category} - insufficient data")
            continue

        # Extract move numbers and times
        win_moves = [mt[0] for mt in results[category]['win']]
        win_times = [mt[1] for mt in results[category]['win']]
        loss_moves = [mt[0] for mt in results[category]['loss']]
        loss_times = [mt[1] for mt in results[category]['loss']]

        # 1. Create histograms of time spent
        plt.figure(figsize=(14, 8))
        bins = np.linspace(0, min(max(max(win_times), max(loss_times)) * 1.1, 120), 30)
        plt.hist(win_times, bins=bins, alpha=0.5, color='green', label='Wins')
        plt.hist(loss_times, bins=bins, alpha=0.5, color='red', label='Losses')
        plt.xlabel('Time Spent (seconds)')
        plt.ylabel('Frequency')
        plt.title(f'Distribution of Time Spent per Move - {category.capitalize()} Games')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig(f"{category}_time_histogram.png")
        plt.close()
        print(f"Histogram saved as {category}_time_histogram.png")

        # 2. Create box plots for time comparison
        plt.figure(figsize=(10, 8))
        box_data = [win_times, loss_times]
        plt.boxplot(box_data, labels=['Wins', 'Losses'], patch_artist=True,
                   boxprops=dict(facecolor='lightblue', color='blue'),
                   whiskerprops=dict(color='blue'),
                   capprops=dict(color='blue'),
                   medianprops=dict(color='red'))
        plt.ylabel('Time Spent (seconds)')
        plt.title(f'Box Plot of Time Spent per Move - {category.capitalize()} Games')
        plt.grid(True, alpha=0.3)
        plt.ylim(0, min(np.percentile(win_times + loss_times, 95) * 1.2, 120))  # Cap at 120 seconds but show 95th percentile
        plt.savefig(f"{category}_time_boxplot.png")
        plt.close()
        print(f"Box plot saved as {category}_time_boxplot.png")

        # 3. Create violin plots
        plt.figure(figsize=(10, 8))
        positions = [1, 2]
        violin_parts = plt.violinplot([win_times, loss_times], positions, showmeans=True, showmedians=True)
        # Customize violin appearance
        for pc in violin_parts['bodies']:
            pc.set_facecolor('lightblue')
            pc.set_alpha(0.7)
        violin_parts['cmeans'].set_color('red')
        violin_parts['cmedians'].set_color('black')
        plt.xticks(positions, ['Wins', 'Losses'])
        plt.ylabel('Time Spent (seconds)')
        plt.title(f'Violin Plot of Time Spent per Move - {category.capitalize()} Games')
        plt.grid(True, alpha=0.3)
        plt.ylim(0, min(np.percentile(win_times + loss_times, 95) * 1.2, 120))
        plt.savefig(f"{category}_time_violin.png")
        plt.close()
        print(f"Violin plot saved as {category}_time_violin.png")

        # 4. Create heatmap of move number vs time spent
        # Group data by move number
        max_move = max(max(win_moves), max(loss_moves))

        # Initialize arrays for the heatmaps
        win_move_times = [[] for _ in range(max_move + 1)]
        loss_move_times = [[] for _ in range(max_move + 1)]

        # Group times by move number
        for move, time in results[category]['win']:
            if move <= max_move:
                win_move_times[move].append(time)

        for move, time in results[category]['loss']:
            if move <= max_move:
                loss_move_times[move].append(time)

        # Calculate average time per move (avoiding division by zero)
        win_avg_times = [np.mean(times) if times else 0 for times in win_move_times]
        loss_avg_times = [np.mean(times) if times else 0 for times in loss_move_times]

        # Create heatmap for wins
        plt.figure(figsize=(14, 6))
        plt.subplot(1, 2, 1)
        # Only include moves with data
        valid_moves = [i for i, times in enumerate(win_move_times) if times]
        if valid_moves:
            plt.bar(valid_moves, [win_avg_times[m] for m in valid_moves], color='green', alpha=0.7)
            plt.xlabel('Move Number')
            plt.ylabel('Average Time (seconds)')
            plt.title(f'Avg Time per Move - {category.capitalize()} Wins')
            plt.grid(True, alpha=0.3)
            plt.ylim(0, min(max([win_avg_times[m] for m in valid_moves if win_avg_times[m]]) * 1.2, 120))

        # Create heatmap for losses
        plt.subplot(1, 2, 2)
        valid_moves = [i for i, times in enumerate(loss_move_times) if times]
        if valid_moves:
            plt.bar(valid_moves, [loss_avg_times[m] for m in valid_moves], color='red', alpha=0.7)
            plt.xlabel('Move Number')
            plt.ylabel('Average Time (seconds)')
            plt.title(f'Avg Time per Move - {category.capitalize()} Losses')
            plt.grid(True, alpha=0.3)
            plt.ylim(0, min(max([loss_avg_times[m] for m in valid_moves if loss_avg_times[m]]) * 1.2, 120))

        plt.tight_layout()
        plt.savefig(f"{category}_move_heatmap.png")
        plt.close()
        print(f"Heatmap saved as {category}_move_heatmap.png")

### Main Execution Block
This is where the script begins execution. It checks for available JSON files, processes each game, and generates visualizations and statistical reports if it is enabled in the main function definition.

In [10]:
if __name__ == '__main__': # To be able to import functions if needed.

    # List of JSON files to process
    json_files = [
        "2023-02.json", "2023-03.json", "2023-04.json", "2023-05.json",
        "2023-06.json", "2023-07.json", "2023-08.json", "2023-09.json",
        "2023-10.json", "2023-11.json", "2023-12.json",
        "2024-01.json", "2024-02.json", "2024-03.json", "2024-04.json",
        "2024-05.json", "2024-06.json", "2024-07.json", "2024-08.json",
        "2024-09.json", "2024-10.json", "2024-11.json", "2024-12.json",
        "2025-01.json", "2025-02.json", "2025-03.json", "2025-04.json"
    ]

    # Check if at least one JSON file exists
    existing_files = [f for f in json_files if os.path.exists(f)]
    if not existing_files:
        print("None of the specified JSON files found. Exiting.")
    else:
        main(json_files)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  - White player: {'rating': 908, 'result': 'win', '@id': 'https://api.chess.com/pub/player/mreko0', 'username': 'MrEKO0', 'uuid': '8008d55a-a521-11ed-adc8-f9403c3e5715'}
  - Black player: {'rating': 907, 'result': 'timeout', '@id': 'https://api.chess.com/pub/player/ariasarias924', 'username': 'ariasarias924', 'uuid': 'b591376c-38d1-11eb-ad3d-b71d6ef5c1c4'}
  - Found MrEKO0 as white player
  - Game ended at: 1687290481
  - White player: {'rating': 880, 'result': 'timeout', '@id': 'https://api.chess.com/pub/player/edoggy25', 'username': 'edoggy25', 'uuid': '86864666-8aab-11eb-9d97-7bdd081acdc8'}
  - Black player: {'rating': 916, 'result': 'win', '@id': 'https://api.chess.com/pub/player/mreko0', 'username': 'MrEKO0', 'uuid': '8008d55a-a521-11ed-adc8-f9403c3e5715'}
  - Found MrEKO0 as black player
  - Game ended at: 1687290612
  - White player: {'rating': 926, 'result': 'win', '@id': 'https://api.chess.com/pub/player/arthur_thi', 'username': 

<ipython-input-9-ce6b8b8f90d8>:35: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(box_data, labels=['Wins', 'Losses'], patch_artist=True,


Violin plot saved as all_time_violin.png
Heatmap saved as all_move_heatmap.png
Histogram saved as blitz_time_histogram.png
Box plot saved as blitz_time_boxplot.png


<ipython-input-9-ce6b8b8f90d8>:35: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(box_data, labels=['Wins', 'Losses'], patch_artist=True,


Violin plot saved as blitz_time_violin.png
Heatmap saved as blitz_move_heatmap.png
Histogram saved as bullet_time_histogram.png
Box plot saved as bullet_time_boxplot.png


...

Violin plot saved as bullet_time_violin.png
Heatmap saved as bullet_move_heatmap.png
Histogram saved as rapid_time_histogram.png
Box plot saved as rapid_time_boxplot.png


<ipython-input-9-ce6b8b8f90d8>:35: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(box_data, labels=['Wins', 'Losses'], patch_artist=True,


Violin plot saved as rapid_time_violin.png
Heatmap saved as rapid_move_heatmap.png
Histogram saved as daily_time_histogram.png
Box plot saved as daily_time_boxplot.png


<ipython-input-9-ce6b8b8f90d8>:35: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(box_data, labels=['Wins', 'Losses'], patch_artist=True,


Violin plot saved as daily_time_violin.png
Heatmap saved as daily_move_heatmap.png


Once the execution of the script is complete, the EDA plots/graphs will be saved to the same directory as this script.